# Intro
This notebook shows how to use the freesound API to retrieve sounds to use in ML models. This notebook assumes you have followed the instructions in the github repo to set up an account with freesound and have updated the fs_config.py file with your id and secret

Import the required packages

In [13]:
import freesound
from requests_oauthlib import OAuth2Session
import os
import time
import pickle
from requests_ratelimiter import LimiterSession

from config import fs_config

Set configuration variables using fs_config.py

# Authenticate

In [14]:


authorization_base_url = 'https://freesound.org/apiv2/oauth2/authorize/'
token_url = 'https://freesound.org/apiv2/oauth2/access_token/'
client_id = fs_config.fs_cid # grab from fs_config.py
client_secret = fs_config.fs_client_secret # grab from fs_config.py

Authenticate
The code below will open a link in your browser. You must authorize, and then copy the code into the python input window to continue

In [15]:
# do the OAuth dance
oauth = OAuth2Session(client_id)

authorization_url, state = oauth.authorization_url(
    "https://freesound.org/apiv2/oauth2/authorize/"
)
print(f"Please go to {authorization_url} and authorize access.")

authorization_code = input("Please enter the authorization code:")
oauth_token = oauth.fetch_token(
    "https://freesound.org/apiv2/oauth2/access_token/",
    authorization_code,
    client_secret=client_secret,
)

# oauth_token
client = freesound.FreesoundClient()
client.set_token(oauth_token["access_token"], "oauth")

Please go to https://freesound.org/apiv2/oauth2/authorize/?response_type=code&client_id=7yB3RiAb8ipn6BQuwvyn&state=IEkDgb5UMfU72BbT4rVJf3tZCaSx0d and authorize access.


Example of how to get a sound using the API

In [16]:
# Get sound info example
print("Sound info:")
print("-----------")
sound = client.get_sound(96541)
print("Getting sound:", sound.name)
print("Url:", sound.url)
print("Description:", sound.description)
print("Tags:", " ".join(sound.tags))
print()

Sound info:
-----------
Getting sound: coming soon.wav
Url: https://freesound.org/people/tim.kahn/sounds/96541/
Description: For the user "Stealth Inc.", my girlfriend saying "Coming soon". Recorded with an AT2020 microphone into an Apogee Duet and edited with Record.
Tags: request female girl talk vocal voice american woman english speak



# About the data:
For this project, we are going to use data uploaded by the Music Technology Group (MTG) of the Universitat Pompeu Fabra in Barcelona. Each sound is roughly 5 seconds of a single instrument playing either notes or scales with different dynamics. We are going to iterate through the list of sounds uploaded by the user MTG and retrieve information about the sounds

In [17]:
# dir(user_sounds[0])

def fs_download(sound):
    """For a given freesound sound object, download audio file + metadata + analysis"""
    # basic info
    sound_info = sound.as_dict()

    # download sound:
    # get basic info
    # check if sound is in file system
    # downloaded_sounds = [x[0] for x in os.walk("./freesound")]
    downloaded_sounds = []

    for root, dirs, files in os.walk("./freesound"):
        for directory in dirs:
            downloaded_sounds.append(directory)
    # if in file_system:
    if str(sound_info['id']) in downloaded_sounds:
        # r"C:\Users\{}\Documents\file.txt".format(username)
        # skip
        print('folder already created. skipping')
        pass
    else:
        # make directory
        sound_dir = f"./freesound/{sound_info['id']}"
        os.mkdir(sound_dir)
        
        # save basic info
        info_pkl = f"{sound_dir}/sound_metadata.pkl"
        with open(info_pkl, 'wb') as f:
            pickle.dump(sound_info, f)
        # save analysis
        sound_analysis = sound.get_analysis().as_dict()
        analysis_pkl = f"{sound_dir}/sound_analysis.pkl"
        with open(analysis_pkl,'wb') as f:
            pickle.dump(sound_analysis,f)
            
        # download
        name_string = f"{sound_info['name']}.{sound_info['type']}"
        name_string = name_string.replace('\r', '')
        sound.retrieve(directory=sound_dir,name = name_string)
        

Note that freesound API has a rate limit of 60 requests/minute and 5000 requests per 24 hours. So we can download a sample of the sounds in one day

In [18]:
from requests_ratelimiter import LimiterSession

# Apply a rate-limit (59 requests per minute) to all requests
client.session = LimiterSession(per_minute=59)

In [19]:
# for simplicity, lets start with all MTG sounds - very clean, usually single instruments
# search for MTG
user = client.get_user("MTG")
# print(dir(user))
print(user.num_sounds)
user_sounds = user.get_sounds(page_size = 100,fields = "id,name,tags,created,license,type,channels,filesize,bitrate,bitdepth,duration,samplerate,username",
                              sort = 'rating_desc')


8117


We have 8117 sounds uploaded by the user. The sounds are presented in a freesound.pager object. The following code iterates through the pager object to download the sounds.

In [20]:
num_pages = user.num_sounds // 100 + 1
print(num_pages)
# num_pages = 2 # temp for testing

82


In [21]:
start_page = 70

page_num = 1

while page_num < start_page:
    tries = 1
    while tries < 10:
        try:
            print("trying next page")
            user_sounds = user_sounds.next_page()
            page_num +=1
            tries = 10
            print(f'Current page num is {page_num}')
        except freesound.FreesoundException as e:
            print('bad gateway, trying again')
            tries += 1
            time.sleep(3)

while page_num < num_pages:
    for sound in user_sounds:
        print("\t-", sound.name, "by", sound.username, "tags", sound.tags)
        fs_download(sound)
    time.sleep(5)
    tries = 0
    while tries < 10:
        try:
            user_sounds = user_sounds.next_page()
            tries = 10
        except freesound.FreesoundException as e:
            print('bad gateway, trying again')
            tries += 1
            time.sleep(3)
    
    # increment page
    page_num += 1

trying next page
bad gateway, trying again
trying next page
Current page num is 2
trying next page
Current page num is 3
trying next page
Current page num is 4
trying next page
Current page num is 5
trying next page
Current page num is 6
trying next page
bad gateway, trying again
trying next page
Current page num is 7
trying next page
Current page num is 8
trying next page
Current page num is 9
trying next page
Current page num is 10
trying next page
Current page num is 11
trying next page
Current page num is 12
trying next page
Current page num is 13
trying next page
Current page num is 14
trying next page
Current page num is 15
trying next page
Current page num is 16
trying next page
Current page num is 17
trying next page
Current page num is 18
trying next page
bad gateway, trying again
trying next page
Current page num is 19
trying next page
bad gateway, trying again
trying next page
Current page num is 20
trying next page
Current page num is 21
trying next page
Current page num is

In [22]:
# audit
# next steps:
# add relevant directories to gitignore
# add git ignore file to github dknapp17/audio_ML
# manually add config and notebooks and requirements.txt to github
# write out workflow for freesound API and running freesound notebook
# document notebook, add cool visuals
# resubmit to github